In [23]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

Function to fix the columns names

In [24]:
def fix_column_names(df):
    for index, column in enumerate(df.columns):
        if '/' in column:
            names = column.split('/')
            first = names[0].strip()
            second = names[0].strip() + " " + names[1].strip()
            
            df.rename(columns={column: first}, inplace=True)
            df.rename(columns={df.columns[index + 1]: second}, inplace=True)

    df.info()
    
    return df

Function to normalize the data in 90 minutes.

In [25]:
def norm_90(df, cols):
    
    # Sum of given columns
    academy_90 = df[cols].sum()

    # Create a column for 90m played
    academy_90["90s"] = int(academy_90["Minutes played"]) / 90

    academy_90 = pd.DataFrame(academy_90).T

    columns_to_divide = cols[1:]

    # Divide these columns by the values in the '90s' column
    academy_90[columns_to_divide] = academy_90[columns_to_divide].div(academy_90['90s'], axis=0)

    # Drop the 90s column
    academy_90.drop("90s", axis=1, inplace=True)

    # Rename the columns to per 90'
    for i in range(1, len(academy_90.columns)):
        academy_90.columns.values[i] = academy_90.columns[i] + '/90'
        
    return academy_90

Get data for academy, first team, and national team

Import Cremaschi data into dataframe

In [26]:
cremaschi_df = pd.read_excel("USA Data/Player stats B. Cremaschi.xlsx", )

cremaschi_df.head(3)

,Match,Competition,Date,Position,Minutes played,Total actions / successful,Unnamed: 6,Goals,Assists,Shots / on target,Unnamed: 10,xG,Passes / accurate,Unnamed: 13,Long passes / accurate,Unnamed: 15,Crosses / accurate,Unnamed: 17,Dribbles / successful,Unnamed: 19,Duels / won,Unnamed: 21,Aerial duels / won,Unnamed: 23,Interceptions,Losses / own half,Unnamed: 26,Recoveries / opp. half,Unnamed: 28,Yellow card,Red card,Defensive duels / won,Unnamed: 32,Loose ball duels / won,Unnamed: 34,Sliding tackles / successful,Unnamed: 36,Clearances,Fouls,Yellow cards,Red cards,Shot assists,Offensive duels / won,Unnamed: 43,Touches in penalty area,Offsides,Progressive runs,Fouls suffered,Through passes / accurate,Unnamed: 49,xA,Second assists,Passes to final third / accurate,Unnamed: 53,Passes to penalty area / accurate,Unnamed: 55,Received passes,Forward passes / accurate,Unnamed: 58,Back passes / accurate,Unnamed: 60
0,Nashville SC - Inter Miami 1:2,United States. MLS,2024-06-30,"LCMF3, RW",99,73,46,0,0,2,0,0.33,43,39,0,0,1,1,3,1,16,6,1,1,0,8,2,6,2,0,0,3,2,2,0,0,0,0,0,0,0,2,10,3,3,1,1,0,0,0,0.02,0,3,3,2,2,31,12,10,14,13
1,Inter Miami - Columbus Crew 2:1,United States. MLS,2024-06-20,"RCMF, RW",98,63,42,0,0,1,0,0.02,42,35,2,1,1,1,0,0,15,7,1,0,5,13,8,9,2,0,0,5,4,2,1,1,1,0,0,0,0,1,7,2,1,0,0,0,0,0,0.10,0,5,3,1,1,27,12,9,7,7
2,Philadelphia Union - Inter Miami 1:2,United States. MLS,2024-06-16,"LCMF, LCMF3",98,77,45,0,1,3,0,0.31,43,36,5,3,0,0,1,0,23,9,4,2,3,12,4,4,3,0,0,2,2,3,2,0,0,2,0,0,0,1,14,3,4,0,2,0,0,0,0.28,0,7,5,2,2,35,12,10,11,10


Clean the column names

In [27]:
cremaschi_df = fix_column_names(cremaschi_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 61 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Match                            74 non-null     object 
 1   Competition                      74 non-null     object 
 2   Date                             74 non-null     object 
 3   Position                         74 non-null     object 
 4   Minutes played                   74 non-null     int64  
 5   Total actions                    74 non-null     int64  
 6   Total actions successful         74 non-null     int64  
 7   Goals                            74 non-null     int64  
 8   Assists                          74 non-null     int64  
 9   Shots                            74 non-null     int64  
 10  Shots on target                  74 non-null     int64  
 11  xG                               74 non-null     float64
 12  Passes                  

We want to create two different df, one for the academy and the other for the first team

In [28]:
cremaschi_academy_df = cremaschi_df[cremaschi_df["Match"].str.contains("Inter Miami II", na=False)]
cremaschi_academy_df.reset_index(inplace=True, drop=True)

cremaschi_firstteam_df = cremaschi_df[~cremaschi_df['Match'].str.contains('Inter Miami II', na=False) & ~cremaschi_df['Match'].str.contains('United States', na=False)]
cremaschi_firstteam_df.reset_index(inplace=True, drop=True)

cremaschi_nationalteam_df = cremaschi_df[cremaschi_df["Match"].str.contains("United States", na=False)]
cremaschi_nationalteam_df.reset_index(inplace=True, drop=True)

Calculate metrics to 90 minutes

In [29]:
cols = ['Minutes played',
        'Total actions', 'Total actions successful', 'Goals', 'Assists',
        'Shots', 'Shots on target', 'xG', 'Passes', 'Passes accurate',
        'Long passes', 'Long passes accurate', 'Crosses', 'Crosses accurate',
        'Dribbles', 'Dribbles successful', 'Duels', 'Duels won', 'Aerial duels',
        'Aerial duels won', 'Interceptions', 'Losses', 'Losses own half',
        'Recoveries', 'Recoveries opp. half', 'Yellow card', 'Red card',
        'Defensive duels', 'Defensive duels won', 'Loose ball duels',
        'Loose ball duels won', 'Sliding tackles', 'Sliding tackles successful',
        'Clearances', 'Fouls', 'Yellow cards', 'Red cards', 'Shot assists',
        'Offensive duels', 'Offensive duels won', 'Touches in penalty area',
        'Offsides', 'Progressive runs', 'Fouls suffered', 'Through passes',
        'Through passes accurate', 'xA', 'Second assists',
        'Passes to final third', 'Passes to final third accurate',
        'Passes to penalty area', 'Passes to penalty area accurate',
        'Received passes', 'Forward passes', 'Forward passes accurate',
        'Back passes', 'Back passes accurate']

cremaschi_academy_df = norm_90(cremaschi_academy_df, cols)
cremaschi_firstteam_df = norm_90(cremaschi_firstteam_df, cols)
cremaschi_nationalteam_df = norm_90(cremaschi_nationalteam_df, cols)

In [30]:
cremaschi_academy_df

,Minutes played,Total actions/90,Total actions successful/90,Goals/90,Assists/90,Shots/90,Shots on target/90,xG/90,Passes/90,Passes accurate/90,Long passes/90,Long passes accurate/90,Crosses/90,Crosses accurate/90,Dribbles/90,Dribbles successful/90,Duels/90,Duels won/90,Aerial duels/90,Aerial duels won/90,Interceptions/90,Losses/90,Losses own half/90,Recoveries/90,Recoveries opp. half/90,Yellow card/90,Red card/90,Defensive duels/90,Defensive duels won/90,Loose ball duels/90,Loose ball duels won/90,Sliding tackles/90,Sliding tackles successful/90,Clearances/90,Fouls/90,Yellow cards/90,Red cards/90,Shot assists/90,Offensive duels/90,Offensive duels won/90,Touches in penalty area/90,Offsides/90,Progressive runs/90,Fouls suffered/90,Through passes/90,Through passes accurate/90,xA/90,Second assists/90,Passes to final third/90,Passes to final third accurate/90,Passes to penalty area/90,Passes to penalty area accurate/90,Received passes/90,Forward passes/90,Forward passes accurate/90,Back passes/90,Back passes accurate/90
0,1181.0,68.204911,40.160881,0.381033,0.152413,1.066892,0.45724,0.087638,32.692633,26.977138,1.676545,0.990686,0.685859,0.304826,3.048264,1.676545,23.471634,11.354784,1.905165,0.990686,2.514818,9.602032,4.115157,6.553768,2.133785,9.068586,0.0,10.592718,6.096528,2.819644,1.066892,0.22862,0.152413,0.762066,1.600339,0.304826,0.0,0.914479,8.154107,3.200677,1.371719,0.0,1.524132,1.295512,0.609653,0.152413,0.117358,0.152413,4.115157,2.895851,1.295512,0.533446,23.624047,10.821338,7.773074,5.563082,4.877223


Import Ricardo Pepi data into dataframe

In [35]:
pepi_df = pd.read_excel('Usa Data/Player stats R. Pepi.xlsx')
pepi_df

,Match,Competition,Date,Position,Minutes played,Total actions / successful,Unnamed: 6,Goals,Assists,Shots / on target,Unnamed: 10,xG,Passes / accurate,Unnamed: 13,Long passes / accurate,Unnamed: 15,Crosses / accurate,Unnamed: 17,Dribbles / successful,Unnamed: 19,Duels / won,Unnamed: 21,Aerial duels / won,Unnamed: 23,Interceptions,Losses / own half,Unnamed: 26,Recoveries / opp. half,Unnamed: 28,Yellow card,Red card,Defensive duels / won,Unnamed: 32,Loose ball duels / won,Unnamed: 34,Sliding tackles / successful,Unnamed: 36,Clearances,Fouls,Yellow cards,Red cards,Shot assists,Offensive duels / won,Unnamed: 43,Touches in penalty area,Offsides,Progressive runs,Fouls suffered,Through passes / accurate,Unnamed: 49,xA,Second assists,Passes to final third / accurate,Unnamed: 53,Passes to penalty area / accurate,Unnamed: 55,Received passes,Forward passes / accurate,Unnamed: 58,Back passes / accurate,Unnamed: 60
0,Panama - United States 2:1,South America. CONMEBOL Copa America,2024-06-28,CF,23,10,4,0,0,1,1,0.45,3,3,0,0,0,0,0,0,5,0,4,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0.00,0,0,0,0,0,2,0,0,1,1
1,United States - Bolivia 2:0,South America. CONMEBOL Copa America,2024-06-24,CF,29,19,13,0,0,6,4,1.63,8,7,0,0,0,0,0,0,3,2,0,0,0,2,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,2,1,6,0,0,1,0,0,0.00,0,1,1,1,1,9,3,2,1,1
2,United States - Brazil 1:1,World. Friendlies,2024-06-13,CF,67,27,15,0,0,0,0,0.00,13,10,0,0,0,0,0,0,9,5,3,0,2,4,2,4,2,0,0,3,3,2,1,0,0,0,1,0,0,0,1,1,2,0,0,0,0,0,0.00,0,2,1,0,0,3,2,1,6,6
3,PSV - RKC Waalwijk 3:1,Netherlands. Eredivisie,2024-05-19,LAMF,27,11,7,0,0,1,0,0.00,9,6,0,0,0,0,1,1,1,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0.00,0,0,0,0,0,7,0,0,6,5
4,Fortuna Sittard - PSV 1:1,Netherlands. Eredivisie,2024-05-12,CF,29,13,11,0,0,1,1,0.05,9,8,0,0,0,0,0,0,3,2,0,0,1,2,1,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0.00,0,0,0,0,0,8,0,0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,North Texas - Forward Madison 1:0,United States. USL League 1,2019-04-14,CF,97,48,24,1,0,7,1,1.10,24,17,0,0,2,0,1,0,9,5,2,1,0,11,2,3,2,0,0,4,3,1,1,0,0,0,0,0,0,1,2,0,5,0,1,0,1,0,0.11,0,0,0,3,0,19,3,1,10,10
192,North Texas - Chattanooga Red Wolves 3:2,United States. USL League 1,2019-03-31,"CF, RAMF",93,70,40,3,0,8,5,2.80,30,23,1,1,0,0,3,3,21,12,6,2,1,14,0,4,3,0,0,9,6,2,1,0,0,1,0,0,0,0,4,3,13,0,0,0,1,0,0.00,0,4,3,2,1,29,6,4,8,7
193,United States U17 - Brazil U17 1:1,World. Nike International Friendlies,2018-12-02,CF,65,37,13,0,0,2,1,1.05,12,8,1,0,0,0,0,0,15,4,3,0,1,5,1,1,1,0,0,5,1,3,1,0,0,0,1,0,0,1,4,2,2,1,0,1,0,0,0.00,0,1,0,0,0,3,3,0,2,1
194,United States U17 - Turkey U17 1:0,World. Nike International Friendlies,2018-12-01,CF,97,69,34,1,0,4,1,0.97,28,21,1,1,0,0,3,2,27,12,2,1,3,12,3,4,4,0,0,5,3,2,0,0,0,0,2,0,0,0,18,8,5,0,2,1,1,0,0.00,0,5,5,0,0,16,8,6,7,5


In [33]:
pepi_u17_df = pepi_df[pepi_df["Match"].str.contains("United States U17", na=False)]
pepi_u17_df.reset_index(inplace=True, drop=True)

pepi_PSV_df = pepi_df[pepi_df["Match"].str.contains("PSV", na=False)]
pepi_PSV_df.reset_index(inplace=True, drop=True)

pepi_USA_df = pepi_df[~pepi_df['Match'].str.contains('United States', na=False) & ~pepi_df['Match'].str.contains('U17', na=False)]
pepi_USA_df.reset_index(inplace=True, drop=True)

pepi_groningen_df = pepi_df[pepi_df["Match"].str.contains("Groningen", na=False)]
pepi_groningen_df.reset_index(inplace=True, drop=True)

pepi_northtexas_df = pepi_df[pepi_df["Match"].str.contains("North Texas", na=False)]
pepi_northtexas_df.reset_index(inplace=True, drop=True)

pepi_dallas_df = pepi_df[pepi_df["Match"].str.contains("Dallas", na=False)]
pepi_dallas_df.reset_index(inplace=True, drop=True)

array(['South America. CONMEBOL Copa America', 'World. Friendlies',
       'Netherlands. Eredivisie',
       'N/C America. CONCACAF Nations League - League A',
       'Europe. UEFA Champions League', 'Netherlands. KNVB Beker',
       'Netherlands. Eerste Divisie', 'Netherlands. Super Cup',
       'World. Club Friendlies', 'Germany. Bundesliga',
       'N/C America. CONCACAF World Cup Qualifiers', 'United States. MLS',
       'United States. USL League 1', 'World. FIFA U17 World Cup',
       'World. U17 National Team Friendlies',
       'World. U18 National Team Friendlies',
       'United States. US Open Cup', 'N/C America. CONCACAF U17',
       'World. Nike International Friendlies'], dtype=object)